In [1]:
import torch
from train import train_loop
from data_module import EEGDataModule
from model import ViTransformer, LSTM, ConvLSTM, DeepConvNet

/home/danny/miniconda3/envs/rl/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
device = torch.device(0)

In [4]:
from preprocessing import mean_max_subsampling

In [5]:
train_params = {
    "train_epochs": 10,
    "accumulate_grad_batches": 1,
    "test_size": 0.2,
    "random_state": 42,
    "data_dir": "../../Data/",
    "train_batch_size": 64,
    "eval_batch_size": 1
}

In [6]:
dataset = EEGDataModule(args=train_params)

In [7]:
dataset.setup(transform=None)

INFO:data_module:Training data shape: (1692, 22, 1000)
INFO:data_module:Training labels shape: (1692,)


In [8]:
train, valid = dataset.train_dataloader(), dataset.val_dataloader()

INFO:data_module:loaded 1692 train data instances
INFO:data_module:loaded 423 train data instances


In [9]:
model = DeepConvNet()

/home/danny/miniconda3/envs/rl/lib/python3.7/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [10]:
from torchinfo import summary
# Only uses outputs of modules.
print(summary(model))

Layer (type:depth-idx)                   Param #
DeepConvNet                              --
├─Conv2d: 1-1                            520
├─Conv2d: 1-2                            8,820
├─MaxPool1d: 1-3                         --
├─Conv1d: 1-4                            5,025
├─Conv1d: 1-5                            12,550
├─BatchNorm1d: 1-6                       50
├─BatchNorm1d: 1-7                       100
├─BatchNorm1d: 1-8                       40
├─Dropout: 1-9                           --
├─LazyLinear: 1-10                       --
├─ELU: 1-11                              --
Total params: 27,105
Trainable params: 27,105
Non-trainable params: 0


In [11]:
loss_hist, acc_hist, val_loss_hist, val_acc_hist = train_loop(model, train, valid, device, weight_decay=1e-5)

100%|██████████████████████████████| 100/100 [01:02<00:00,  1.59it/s, acc=0.891, val_acc=0.596]
